# Estimateur de Ridge pour restreindre la variance :
>__Auteurs__:  Joly Julien , Mohamed Sahardid et Anas El Benna

Avant toutes choses, nous commençons par importer les packages qui vont nous êtres nécessaires et nous récupérons les données grâce au code de nos camarades Arielle Gantelet, Vitus Kirchberger et Samuel Valiquette.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import RidgeCV, Ridge
from pathlib import Path
from statsmodels.formula.api import ols
import sys

sys.path.append("./codes")
plt.rcParams.update({'figure.max_open_warning': 25})
%matplotlib notebook
directory = Path.cwd()
directory = directory.joinpath('prebuiltimages')


def my_saving_display(fig, directory, image_name, imageformat="pdf",
                      saving=False):

    """"Saving with personal function."""
    if saving is True:
        fig.savefig(directory.joinpath(image_name + '.' + imageformat),
                    format=imageformat)


In [2]:
from download import download
url = "http://josephsalmon.eu/enseignement/datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
path_target = "datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=True)


RuntimeError: Error while fetching file http://josephsalmon.eu/enseignement/datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv. Dataset fetching aborted.
Error: <urlopen error [Errno -3] Temporary failure in name resolution>

In [3]:
occ = pd.read_csv(path_target)
occ['date'] = pd.to_datetime(occ['date_debut']).dt.to_period('M')


/home/jo/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1172: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  "will drop timezone information.", UserWarning)


Maintenant que nos données sont récupérées nous créons une liste "ville" où l'utilisateur entre les villes qu'il veut étudier. Remarquons que la commande isin() permet de vérifier l'existence de ces villes dans nos données. 
Nous définissons comme date le mois de juillet 2018 de manière arbitraire et nous allant regardé les relevés d'ozone.

In [4]:
# ville = occ.nom_com.unique() #Pour récupérérer toutes les villes de
# notre data fame.
ville = 'MONTPELLIER', 'TOULOUSE', 'AGDE'

df = occ[occ['nom_com'].isin(ville)]
df = df[df['polluant'] == 'O3']

df.date.unique()

df = df[df.date == '2018-07']
df = df[['nom_com', 'valeur_originale']]


Nous récupérons ensuite sous forme d'array les relevés d'ozone des villes choisies ainsi que leur nombre (car les villes n'ont pas forcément le même nombre d'observations.  

In [5]:
valeur_o3 = []
len_villes = []

for i in range(len(ville)):

    o3_i = df[df['nom_com'] == ville[i]].values[:, 1]

    # valeurs de O3 des villes choisies
    valeur_o3 = np.append(valeur_o3, o3_i)

    # Nombre d'observations de chaque ville
    len_villes = np.append(len_villes, o3_i.shape)


Nous recodons ensuite nos villes en variables qualitatives de manière à pouvoir effectuer notre régression Ridge.

In [6]:
X = np.zeros((int(np.sum(len_villes)), len(ville)))

for i in range(len(ville)):
    X[int(np.sum(len_villes[0:i])):int(np.sum(len_villes[0:i+1])),
      i] = np.ones(int(np.sum(len_villes[i])))


Nous définissons maintenant les valeurs du critère alpha de pénalisation que nous allons parcourir.

In [7]:
eps = 1e-6
alpha_max = 1e4
n_alphas = 100
alphas = np.logspace(np.log10(alpha_max * eps),
                     np.log10(alpha_max), num=n_alphas)


Nous réalisons maintenant la régression Ridge suivant ces valeurs de alpha et nous stockons les valeurs obtenues dans lamatrice ridge.

In [8]:
ridge = np.zeros((len(alphas), len(ville)))

for j in range(len(ville)):
    y_predict = np.zeros(len(ville))
    y_predict[j] = 1

    for i in range(len(alphas)):

        # Modele Ridge avec Validation croisée
        model_ridge = Ridge(alphas[i])

        # Applique le modèle sur le training set
        model_ridge.fit(X, valeur_o3)

        pred_train = model_ridge.predict(X)

        ridge[i, j] = model_ridge.predict([y_predict])


Nous réalisons maintenant une fonction permettant, à l'aide d'un paramètre alpha, qui contient la valeur maximale, la valeur minimale et le pas voulu, de voir l'évolution de l'estimateur ridge graphiquement à l'aide d'un bouton modifiant la valeur du critère alpha du Ridge.

In [9]:
def iteration_ridge_alpha(logalpha):
    np.random.seed(0)
    current_palette = sns.color_palette()
    fig = sns.catplot(y="valeur_originale", x="nom_com", data=df,
                      jitter='0.05', legend_out=False, order=list(ville),
                      height=8)

    plt.axhline(y=np.mean(valeur_o3), xmin=0, xmax=len(ville), ls='--',
                linewidth=3, color='grey')
    plt.text(len(ville)-0.3, np.mean(valeur_o3), '$\\bar y$', color='grey',
             verticalalignment='bottom', horizontalalignment='right',
             fontsize=25)

    for i in range(len(ville)):

        y_predict = np.zeros(len(ville))
        y_predict[i] = 1
        model_ridge = Ridge(np.exp(logalpha))
        model_ridge.fit(X, valeur_o3)
        ridge = model_ridge.predict([y_predict])

        plt.scatter(i, np.mean(valeur_o3[int(np.sum(len_villes[0:i])):
                    int(np.sum(len_villes[0:i+1]))]), marker='_', lw=10,
                    color=current_palette[i], s=600)

        plt.text(i + 0.2, np.mean(valeur_o3[int(np.sum(len_villes[0:i])):
                 int(np.sum(len_villes[0:i+1]))]), '$\\bar y _{}$'.format(i),
                 verticalalignment='bottom', horizontalalignment='left',
                 color=current_palette[i], fontsize=20)

        plt.scatter(i, ridge, s=400, marker='_',
                    color=current_palette[len(ville)+i], lw=5,
                    label='ridge_{}'.format(ville[i]))

    plt.legend(loc='upper left', bbox_to_anchor=(1., .9))
    plt.tight_layout()
    plt.title('Evolution de l estimateur Ridge suivant la valeur \n'
              'du critère de pénalisation alpha')


Nous illustrons cette fonction ci-dessous.

In [10]:
import ipywidgets as ip
ip.interact(iteration_ridge_alpha, logalpha=(-10, 10, 0.1))


interactive(children=(FloatSlider(value=0.0, description='logalpha', max=10.0, min=-10.0), Output()), _dom_cla…

<function __main__.iteration_ridge_alpha(logalpha)>